<h1 style="text-align: center;">MAGIC CUBE VISUALIZATION</h1>
<h2 style="text-align: center;">RUN EACH CELL ONE BY ONE!</h2>


In [ ]:
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

<h2 style="text-align: center;">(ALL) HILL-CLIMBING VISUALIZATION</h2>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
import matplotlib.colors as mcolors

directory = Path('log')
folder = [d.name for d in directory.iterdir() if d.is_dir() and d.name.startswith('HCSA')]

logPath = widgets.Dropdown(
    options=folder,
    value=None,
    description='FOLDER'
)

option_output = widgets.Output()

option = widgets.Dropdown(
    options=['PROCESS', 'OBJECTIVE FUNCTION'],
    value=None,
    description='OPSI',
    disabled=True
)

slider_output = widgets.Output()
chart_output = widgets.Output()

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Slider:',
    style={'description_width': 'initial'},
    continuous_update=True
)

cube, score, status, swap = [], [], [], []

def LP_drp(change):
    if change['new'] is not None:
        option.disabled = False
        clear_data()
        load_data(change["new"])
        update_slider(len(cube))
    else:
        option.disabled = True

def clear_data():
    cube.clear()
    status.clear()
    swap.clear()
    score.clear()

def load_data(folder_name):
    with open(f'log/{folder_name}/cube.txt', 'r') as cubeFile:
        for line in cubeFile:
            cube.append(list(map(int, line.split())))

    with open(f'log/{folder_name}/status.txt', 'r') as statusFile:
        for line in statusFile:
            status.append(list(map(float, line.split())))

    with open(f'log/{folder_name}/swap.txt', 'r') as swapFile:
        for line in swapFile:
            swap.append(list(map(int, line.split())))

    with open(f'log/{folder_name}/score.txt', 'r') as scoreFile:
        for line in scoreFile:
            score.append(list(map(int, line.split())))

def update_slider(cube_length):
    slider.min = 0
    slider.max = cube_length - 1
    slider.value = 0

def O_drp(change):
    with option_output:
        clear_output(wait=True)
        slider.layout.visibility = 'hidden'

        if change['new'] == 'PROCESS':
            slider.layout.visibility = 'visible'
            display(slider, slider_output)
            slider.observe(on_slider_change, names='value')

        elif change['new'] == 'OBJECTIVE FUNCTION':
            plot_score()

def on_slider_change(change):
    with chart_output:
        clear_output(wait=True)
        print(f"OBJECTIVE FUNCTION SCORE: {score[change['new']]}")
        draw_chart(change['new'])

def plot_score():
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(score, label="Objective Function", color='b', marker='o')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Score')
    ax.set_title('Objective Function Score Over Iterations')
    ax.legend()
    plt.grid(True)
    plt.show()

def draw_chart(slider_value):
    fig, ax = plt.subplots(figsize=(29, 5))
    num_grids, grid_size = 5, 5
    spacing = 0.5

    available_width = fig.get_figwidth()
    available_height = fig.get_figheight()
    total_width = num_grids * grid_size + (num_grids - 1) * spacing
    total_height = grid_size

    square_size = min(available_width / total_width, available_height / total_height)
    offset = (grid_size * square_size) / 2

    status_flat = np.array(status[slider_value]).flatten()
    norm = mcolors.Normalize(vmin=np.min(status_flat), vmax=np.max(status_flat))
    cmap = plt.get_cmap('RdYlGn_r')

    for k in range(num_grids):
        for i in range(grid_size):
            for j in range(grid_size):
                x_offset = k * (grid_size * square_size + spacing)
                num = cube[slider_value][k * 25 + j * 5 + i]
                status_value = status[slider_value][k * 25 + j * 5 + i]
                rgba_color = cmap(norm(status_value))
                rgb_color = rgba_color[:3]

                if num == swap[slider_value][0] or num == swap[slider_value][1]:
                    square = plt.Rectangle(
                        (i * square_size - offset + x_offset, j * square_size - offset),
                        square_size, square_size, linewidth=2, edgecolor='black', facecolor='blue'
                    )
                    ax.add_patch(square)
                    ax.text(
                        i * square_size - offset + x_offset + square_size / 2,
                        j * square_size - offset + square_size / 2,
                        str(num), ha='center', va='center', fontsize=20
                    )
                else:
                    square = plt.Rectangle(
                        (i * square_size - offset + x_offset, j * square_size - offset),
                        square_size, square_size, linewidth=2, edgecolor='black', facecolor=rgb_color
                    )
                    ax.add_patch(square)
                    ax.text(
                        i * square_size - offset + x_offset + square_size / 2,
                        j * square_size - offset + square_size / 2,
                        str(num), ha='center', va='center', fontsize=20
                    )

        ax.text(
            k * (grid_size * square_size + spacing) + square_size * 2.5 - offset,
            3, 'Layer ' + str(k + 1), ha='center', va='bottom', fontsize=20
        )

    ax.set_xlim(-offset, num_grids * (grid_size * square_size + spacing) - spacing - offset)
    ax.set_ylim(-offset, grid_size * square_size - offset)
    ax.axis('off')

    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

logPath.observe(LP_drp, names='value')
option.observe(O_drp, names='value')

display(logPath, option, option_output, slider_output, chart_output)
